# Kullback-Leibler divergence and cross entropy

## Goal: understand it better

[https://en.wikipedia.org/wiki/Kullback–Leibler_divergence](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence)

When I first learned about KL divergence I thought it was so cool. A way to tell how different one distribution is from another! Only later have I realized how important it is in machine learning and statistics in general. Let's dive in.

The general formula for computing the KL divergence is as follows:

$D_\text{KL}(P \parallel Q) = \sum_{x\in\mathcal{X}} P(x) \log\left(\frac{P(x)}{Q(x)}\right).$

In words: the KL divergence is the expected value of $P(x)$ mulitplied by the logarithmic difference of $P(x)$ and $Q(x)$

Let's make a basic example

https://en.wikipedia.org/wiki/Expected_value

https://en.wikipedia.org/wiki/Cross_entropy

https://en.wikipedia.org/wiki/Exponential_function

We use the base 2 logarithm and get our resulting unit in bits: https://en.wikipedia.org/wiki/Units_of_information

Using $e$ would result in nats: https://en.wikipedia.org/wiki/Nat_(unit)

In [12]:
import math

In [182]:
def KL(p, q):
    assert(len(p) == len(q))
    return sum([x*math.log2(x/y) for x,y in zip(p,q)])

In [201]:
P = [0.36, 0.48, 0.16]
Q = [1/3, 1/3, 1/3]

In [184]:
def entropy(X):
    return -sum([x*math.log2(x) for x in X])

def H(X):
    return entropy(X)

In [202]:
entropy([0.5, 0.5]), entropy([0.7,0.5])

(1.0, 0.8602012209808307)

In [203]:
entropy(P), entropy(Q)

(1.4619011889093374, 1.584962500721156)

In [187]:
KL(P,Q)

0.12306131181181895

In [189]:
KL(Q,P)

0.14059785499907884

Combining KL divergence and entropy, we get cross entropy:

In [190]:
H(P) + KL(P,Q)

1.5849625007211563

We can simplify into a single formula:

In [195]:
def cross_entropy(p, q):
    assert(len(p) == len(q))
    return -sum([x*math.log2(y) for x,y in zip(p,q)])

In [196]:
cross_entropy(P,Q)

1.584962500721156

We can also create the _logistic_ cross entropy, which takes a sample and computes the cross entropy!

In [316]:
P = [0.25, 0.25, 0.25, 0.25]
Q = [0.125, 0.125, 0.25, 0.5]

In [223]:
import random

In [338]:
def log_loss(p, q, N):
    assert(len(p) == len(q))
    indices = random.sample(range(len(p)), N)
    return -sum([p[i]*math.log(q[i])+(1-p[i])*math.log(1-q[i]) for i in indices]) / N

In [339]:
log_loss(P,Q,4)

0.6238750462388638

Let's see what happens for different values of N (sample size)

In [349]:
for i in range(1, 4+1):
    average = 0.0
    for _ in range(32):
        average += log_loss(P, Q, i)
    print(i, ':', average/32)

1 : 0.6152160294487287
2 : 0.6242929345898863
3 : 0.623274277642306
4 : 0.6238750462388638


And check it against a known good implementation in TensorFlow :)

In [340]:
import tensorflow as tf
tf.losses.log_loss(P,Q).numpy()

0.6238748

This matches the above!

# 🎉

Let's also try out Hellinger distance! https://en.wikipedia.org/wiki/Hellinger_distance

In [77]:
def HD(p, q):
    assert(len(p) == len(q))
    return (1 / math.sqrt(2)) * math.sqrt(sum([(math.sqrt(x)-math.sqrt(y))**2 for x,y in zip(p,q)]))

In [101]:
HD(P,Q)

0.15049827510763777

In [102]:
HD(Q,P)

0.15049827510763777

It's symmetric! Which is exactly why it was used in [_Word Embeddings through Hellinger PCA_ (PDF)](https://www.aclweb.org/anthology/E14-1051.pdf).